# Protypical Networks

Pytorch implementation

In [1]:
%pylab inline

import os
import logging
# logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logging.basicConfig(level=logging.INFO, format='%(message)s')
# logging.basicConfig(level=logging.INFO)
Logger = logging.getLogger('protonet')
Logger.setLevel(logging.DEBUG)

# import skimage, skimage.io, skimage.transform

Populating the interactive namespace from numpy and matplotlib


In [2]:
import torch                                        # root package
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.
# torch.manual_seed(time.time())

# from torch.utils.data import Dataset, DataLoader    # dataset representation and loading
# import torch.autograd as autograd         # computation graph
# from torch import Tensor                  # tensor node in the computation graph
# import torch.nn as nn                     # neural networks
# import torch.nn.functional as F           # layers, activations and more
# from torch.jit import script, trace       # hybrid frontend decorator and tracing jit

# import torchvision
# from torchvision import datasets, models, transforms
# print(torchvision.__version__)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device {} is available.'.format(device))

use_cuda = True

Device cuda is available.


In [3]:
import fsl

## Load data

In [4]:
datadir = os.path.expanduser('~/Database/Machine_Learning')
workdir = os.getcwd()

outdir = f'{workdir}/outputs'

try:
    os.makedirs(outdir)
except Exception as msg:
    print(msg)
    pass

[Errno 17] File exists: '/home/han/Projects/Few Shot Learning/Notebooks/Prototype_Networks/outputs'


### Demonstration of meta-sampler

In [5]:
# ## On the original dataset
# indir = f'{datadir}/Omniglot/omniglot.original/python/'

# foo1 = nd.from_folder(f'{indir}/images_background')
# foo2 = nd.from_folder(f'{indir}/images_evaluation')

# data_dict = {'images_background': foo1, 'images_evaluation': foo2}
# # print(len(data_dict.keys()))

# ms = fsl.utils.data.Meta_Sampler(data_dict, 0.75, lvl_split=1, lvl_task=2, n_way=20, k_shot=5)

# print(len(ms.trainval_dict.keys()))

# print(len(ms.test_dict.keys()))

In [6]:
## On the fused dataset
indir = f'{datadir}/Omniglot/omniglot.fused/data/'

data_dict = fsl.utils.nested_dict.from_folder(indir)
train_dict, test_dict = fsl.utils.nested_dict.random_split_level(data_dict, 0.75, 0)

train_sampler = fsl.utils.data.Meta_Sampler(train_dict, indir, lvl_task=1, n_way=10, k_shot=5, q_shot=None)
test_sampler = fsl.utils.data.Meta_Sampler(test_dict, indir, lvl_task=1, n_way=10, k_shot=5, q_shot=None)

# test_sampler = train_sampler

## Model construction

In [7]:
import torch.optim as optim

kwargs_imread = dict(invert=True, dim=(20, 20))
model = fsl.models.torch.ProtoNet((1, *kwargs_imread['dim']), use_cuda=use_cuda)

# if use_cuda:
#     model.cuda()
# else:
#     model.cpu()
    
decay_every = 20
learning_rate = 1e-4
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0)
scheduler = optim.lr_scheduler.StepLR(optimizer, decay_every, gamma=0.5)

/home/han/.conda/envs/dev38_pip/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


### Training

In [8]:
maxiter_episode = 100
maxiter_task = 10
maxiter_test = 10
maxiter_sample = 1000

for i in range(maxiter_episode):  # loop over the episode
    Logger.info(f'-----Episode {i+1}/{maxiter_episode}-----')
    # train loop
    model.train()
        
    # a task in meta-learning is equivalent to a mini-batch in normal learning
    for t in range(maxiter_task):        
        # draw a task, use sample_task_local() to generate classes at the local level.
        task = train_sampler.sample_task()        
        
        Loss, Acc = [], []
#         loss_func = None
                
        for n in range(maxiter_sample):  # maxiter_samples <-> mini-batch size            
            # draw support-query samples
            (sa, sl), (qa, ql) = train_sampler.sample_support_query(task, split=True, offclass=False, \
                                                                    **kwargs_imread)                        
            # assert sl==ql, labels will not be used
            # assert len(sa)==len(sl) and len(qa)==len(ql)                            
        
            # compute prototype of features for support
            S = torch.stack([model.forward(x).mean(axis=0) for x in sa])
            # compute features for query 
            Q = model.forward(qa.reshape((-1, *qa.shape[2:])))
            loss, acc = model.loss_acc(Q, S)
#             try:
#                 loss_func += loss
#             except:
#                 loss_func = loss
            
#             if n%10 == 0:
#                 Logger.info(f'Sample {n}/{maxiter_sample}: loss={loss.item():.3e}, acc={acc.item():.3e}')
            
            Loss.append(loss.item())
            Acc.append(acc.item())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        scheduler.step()
        Logger.info(f'Task {t+1}/{maxiter_task}: loss.mean={np.mean(Loss):.3e}, acc.mean={np.mean(Acc):.3e}')
        
#         # Gradient step outside the sample loop is infeasible due to the memory overflow of loss_func
#         optimizer.zero_grad()
#         loss_func.backward()
#         optimizer.step()
    
    # validation loop
#     model.eval()  # same as net.train(False), gives wrong results    
    with torch.no_grad():
        Loss, Acc = [], []
        for t in range(maxiter_test):
            # draw a task
            task = test_sampler.sample_task()        

            # draw support-query samples
            (sa, sl), (qa, ql) = test_sampler.sample_support_query(task, split=True, offclass=False, \
                                                                   **kwargs_imread)                        
            # compute prototype of features for support
            S = torch.stack([model.forward(x).mean(axis=0) for x in sa])
            # compute features for query 
            Q = model.forward(qa.reshape((-1, *qa.shape[2:])))
            loss, acc = model.loss_acc(Q, S)

            Loss.append(loss.item())
            Acc.append(acc.item())

        Logger.info(f'Validation: loss.mean={np.mean(Loss):.3e}, acc.mean={np.mean(Acc):.3e}\n')        

-----Episode 1/100-----
Task 1/10: loss.mean=1.258e-01, acc.mean=9.664e-01
Task 2/10: loss.mean=5.531e-02, acc.mean=9.848e-01
Task 3/10: loss.mean=4.302e-02, acc.mean=9.875e-01
Task 4/10: loss.mean=3.999e-02, acc.mean=9.880e-01
Task 5/10: loss.mean=4.950e-02, acc.mean=9.855e-01
Task 6/10: loss.mean=3.761e-02, acc.mean=9.898e-01
Task 7/10: loss.mean=4.229e-02, acc.mean=9.880e-01
Task 8/10: loss.mean=2.961e-02, acc.mean=9.917e-01
Task 9/10: loss.mean=2.281e-02, acc.mean=9.940e-01
Task 10/10: loss.mean=2.600e-02, acc.mean=9.930e-01
Validation: loss.mean=2.251e+00, acc.mean=3.687e-01

-----Episode 2/100-----
Task 1/10: loss.mean=2.135e-02, acc.mean=9.939e-01
Task 2/10: loss.mean=3.342e-02, acc.mean=9.900e-01
Task 3/10: loss.mean=1.886e-02, acc.mean=9.946e-01
Task 4/10: loss.mean=1.830e-02, acc.mean=9.951e-01
Task 5/10: loss.mean=2.898e-02, acc.mean=9.920e-01
Task 6/10: loss.mean=2.174e-02, acc.mean=9.937e-01
Task 7/10: loss.mean=2.642e-02, acc.mean=9.928e-01
Task 8/10: loss.mean=1.489e-02,

-----Episode 15/100-----
Task 1/10: loss.mean=1.376e-01, acc.mean=9.556e-01
Task 2/10: loss.mean=2.301e-01, acc.mean=9.221e-01
Task 3/10: loss.mean=3.397e-01, acc.mean=8.901e-01
Task 4/10: loss.mean=5.522e-01, acc.mean=8.382e-01
Task 5/10: loss.mean=8.697e-02, acc.mean=9.696e-01
Task 6/10: loss.mean=2.237e-01, acc.mean=9.234e-01
Task 7/10: loss.mean=1.592e-01, acc.mean=9.431e-01
Task 8/10: loss.mean=1.510e-01, acc.mean=9.496e-01
Task 9/10: loss.mean=2.056e-01, acc.mean=9.277e-01
Task 10/10: loss.mean=1.130e-01, acc.mean=9.636e-01
Validation: loss.mean=6.454e-01, acc.mean=7.973e-01

-----Episode 16/100-----
Task 1/10: loss.mean=1.444e-01, acc.mean=9.474e-01
Task 2/10: loss.mean=2.064e-01, acc.mean=9.325e-01
Task 3/10: loss.mean=1.526e-01, acc.mean=9.495e-01
Task 4/10: loss.mean=1.885e-01, acc.mean=9.288e-01
Task 5/10: loss.mean=2.561e-01, acc.mean=9.148e-01
Task 6/10: loss.mean=1.898e-01, acc.mean=9.320e-01
Task 7/10: loss.mean=2.087e-01, acc.mean=9.341e-01
Task 8/10: loss.mean=2.881e-0

Validation: loss.mean=7.132e-01, acc.mean=7.733e-01

-----Episode 29/100-----
Task 1/10: loss.mean=3.588e-01, acc.mean=8.846e-01
Task 2/10: loss.mean=2.197e-01, acc.mean=9.138e-01
Task 3/10: loss.mean=4.137e-01, acc.mean=8.545e-01
Task 4/10: loss.mean=4.080e-01, acc.mean=8.715e-01
Task 5/10: loss.mean=3.162e-01, acc.mean=8.808e-01
Task 6/10: loss.mean=3.746e-01, acc.mean=8.500e-01
Task 7/10: loss.mean=2.655e-01, acc.mean=9.059e-01
Task 8/10: loss.mean=1.882e-01, acc.mean=9.308e-01
Task 9/10: loss.mean=5.763e-01, acc.mean=7.883e-01
Task 10/10: loss.mean=2.056e-01, acc.mean=9.367e-01
Validation: loss.mean=7.269e-01, acc.mean=7.600e-01

-----Episode 30/100-----
Task 1/10: loss.mean=5.565e-01, acc.mean=8.197e-01
Task 2/10: loss.mean=3.260e-01, acc.mean=8.762e-01
Task 3/10: loss.mean=8.898e-01, acc.mean=7.216e-01
Task 4/10: loss.mean=2.957e-01, acc.mean=8.982e-01
Task 5/10: loss.mean=4.104e-01, acc.mean=8.663e-01
Task 6/10: loss.mean=6.305e-01, acc.mean=8.054e-01
Task 7/10: loss.mean=1.640e

Task 10/10: loss.mean=3.297e-01, acc.mean=9.005e-01
Validation: loss.mean=6.431e-01, acc.mean=7.907e-01

-----Episode 43/100-----
Task 1/10: loss.mean=3.323e-01, acc.mean=8.830e-01
Task 2/10: loss.mean=1.328e-01, acc.mean=9.491e-01
Task 3/10: loss.mean=1.929e-01, acc.mean=9.326e-01
Task 4/10: loss.mean=4.046e-01, acc.mean=8.617e-01
Task 5/10: loss.mean=4.825e-01, acc.mean=8.365e-01
Task 6/10: loss.mean=2.849e-01, acc.mean=8.971e-01
Task 7/10: loss.mean=4.576e-01, acc.mean=8.547e-01
Task 8/10: loss.mean=2.070e-01, acc.mean=9.389e-01
Task 9/10: loss.mean=2.881e-01, acc.mean=8.952e-01
Task 10/10: loss.mean=1.709e-01, acc.mean=9.389e-01
Validation: loss.mean=5.825e-01, acc.mean=8.093e-01

-----Episode 44/100-----
Task 1/10: loss.mean=3.924e-01, acc.mean=8.462e-01
Task 2/10: loss.mean=5.241e-01, acc.mean=8.105e-01
Task 3/10: loss.mean=1.950e-01, acc.mean=9.261e-01
Task 4/10: loss.mean=2.053e-01, acc.mean=9.276e-01
Task 5/10: loss.mean=5.844e-01, acc.mean=8.060e-01
Task 6/10: loss.mean=1.966

KeyboardInterrupt: 

In [10]:
model.cpu()
torch.save(model, os.path.join(outdir, 'model_torch.pt'))


## EOF